In [1]:
import ast
import re
import io
import json
import gzip
import numpy as np
import pandas as pd
from google.cloud import storage

In [2]:
pd.set_option('max_colwidth', 999999)
pd.options.mode.copy_on_write = True

In [4]:
bucket_name = "chronos-cron-prod"
source = "chronos-2024-02-26.access.log.gz" # gs uri

# Explicitly use service account credentials by specifying the private key file.
storage_client = storage.Client.from_service_account_json('/content/perqara-data-532572ce4996.json')

# Get the bucket and blob objects
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(source)

# Download the contents of the blob as a string
str_data = io.BytesIO(blob.download_as_string())

In [7]:
# Open gzip into csv
with gzip.open(str_data) as gz:
  # Read compressed file as a file object
  file = gz.read()
  # Decode the byte type into string by utf-8
  blob_decompress = file.decode('utf-8')
  # StringIO object
  s = io.StringIO(blob_decompress)

In [8]:
df = pd.read_json(s, dtype={'event': str, 'level': str, 'message': str, 'timestamp': pd.Timestamp}, nrows=999999999, lines=True)

In [9]:
df

,event,level,message,timestamp
0,disconnecting,info,socket client disconnecting with id: ZrT3fzyJ_PVf-uM2AOCT,2024-02-26 00:00:02
1,connection,info,socket client connected with id: _OOqz1myy0nlefqAAOCd,2024-02-26 00:00:40
2,connection,info,socket client connected with id: FPDLM7qFk3HmpzOqAOCf,2024-02-26 00:00:50
3,disconnecting,info,socket client disconnecting with id: FPDLM7qFk3HmpzOqAOCf,2024-02-26 00:01:25
4,connection,info,socket client connected with id: Dig-THOxumyz3xQ9AOCh,2024-02-26 00:02:15
...,...,...,...,...
33871,nan,info,"{'data': {'client_id': '15697', 'token': '46318|3WFSffwCMYfOQyQxaAsn9T5VAbHk2OGoRtx4AeVe3881b1e4'}, 'event': 'CLIENT:REGISTER', 'socket_id': 'THZ13g_jP1apXXQ4AR68'}",2024-02-26 23:59:46
33872,connection,info,socket client connected with id: 24VzhVZWkyfHklrxAR7J,2024-02-26 23:59:47
33873,connection,info,socket client connected with id: ztJ7VnQcGmvoRFa3AR7K,2024-02-26 23:59:47
33874,nan,info,"{'data': {'client_id': '15697', 'token': '46318|3WFSffwCMYfOQyQxaAsn9T5VAbHk2OGoRtx4AeVe3881b1e4'}, 'event': 'CLIENT:REGISTER', 'socket_id': 'ztJ7VnQcGmvoRFa3AR7K'}",2024-02-26 23:59:47


In [10]:
df = df[df['message'].str.contains("room_key")]

In [11]:
df[['message']]

,message
2076,"{'data': {'booking': {'booking_date': '2024-02-26', 'booking_time': '03:39:00', 'bookingable_id': 13671, 'bookingable_type': 'AvailabilityInstant', 'consultation_id': 18569, 'created_at': '2024-02-26T03:39:39.000000Z', 'duration': 30, 'id': 18411, 'parent_id': 0, 'status': 0, 'updated_at': '2024-02-26T03:39:39.000000Z'}, 'client': {'address': {'address': 'jl birah 1 no 9', 'city_id': 3276, 'city_name': 'DEPOK', 'id': 652, 'postal_code_id': 0, 'status': 1}, 'birth_date': '1995-09-22', 'gender': 'MALE', 'id': 5804, 'is_completed': True, 'name': 'Krisna', 'photo_url': '', 'profile_completions': {'section_validations': [], 'validation_message': 'Data anda belum lengkap', 'validation_percentage': 88}}, 'client_attendance': '', 'description': 'test', 'id': 18569, 'is_client_rated': False, 'last_call': '2024-02-26T03:39:45.000000Z', 'lawyer': {'affidavit_date': '2015-12-31T17:00:00.000000Z', 'agency_city': {'id': 3171, 'name': 'JAKARTA SELATAN'}, 'agency_name': 'Perqara', 'agency_province': {'id': 31, 'name': 'DKI JAKARTA'}, 'avg_rating': 5, 'avg_ratings': '5.0', 'city': {'id': 3171, 'name': 'JAKARTA SELATAN'}, 'description': 'Akun ini bertujuan khusus untuk Survei Klien yang memiliki kendala Teknis pada Platform Perqara. Di luar dari Responden maka akan Kami tolak.', 'educations': [{'degree': 'S1', 'institution_id': 189, 'institution_name': 'Duta Bangsa University'}], 'gender': 'MALE', 'id': 324, 'is_busy': False, 'is_online': True, 'is_probono': True, 'name': 'Perqara Testing', 'photo_url': 'https://storage.googleapis.com/perqara-public/lawyer/photo_1708583998', 'price': 0, 'skill_ids': [2, 3], 'skills': [{'id': 2, 'name': 'Pidana'}, {'id': 3, 'name': 'Perdata'}], 'slug': 'Perqara Testing', 'total_consultations': 0, 'year_exp': 8}, 'lawyer_approved_at': '', 'lawyer_attendance': '', 'room_expired_at': '', 'room_key': 'eyJpdiI6IlVRS2J0eUlFMUl3alpHTWZrUzJwQ3c9PSIsInZhbHVlIjoiS3duMXRud0FWQjhsdEF3STA5RTloWTM3RGZyTHIxb2I2dlo1NUU5RW00bm5ldUNJYVVURkloaUI0cFJKcmkvRjNuVTJmMEFINFk3elE1eVBYWWJEVmVWdy9BdWQyYkhPS2RydURueDJaZlVDL0VqR2RQQnd1L3plNTV0bWJEcmYxOFgybytRaWEvQkNLUzVFeDRpcHhRPT0iLCJtYWMiOiJmYTMxYzhmN2JjYzcyYjE5YzRkNzE4YWZmNzAzMDkwNmMzYmIxMGExZmE3MmY1NmNkOGNiYTZjMmQwYzAyNjcwIiwidGFnIjoiIn0=', 'skill': {'id': 2, 'name': 'Pidana'}, 'status': 'WAITING_FOR_APPROVAL', 'stop_time': '', 'summary': {}, 'summary_sent_at': '', 'total_price': 0, 'waiting_expired_at': '2024-02-26T03:42:45.000000Z'}, 'event': 'LAWYER:NEW_CONSULTATION-TRYING-0'}"
2081,"{'data': {'booking': {'booking_date': '2024-02-26', 'booking_time': '03:39:00', 'bookingable_id': 13671, 'bookingable_type': 'AvailabilityInstant', 'consultation_id': 18569, 'created_at': '2024-02-26T03:39:39.000000Z', 'duration': 30, 'id': 18411, 'parent_id': 0, 'status': 2, 'updated_at': '2024-02-26T03:39:48.000000Z'}, 'client': {'address': {'address': 'jl birah 1 no 9', 'city_id': 3276, 'city_name': 'DEPOK', 'id': 652, 'postal_code_id': 0, 'status': 1}, 'birth_date': '1995-09-22', 'gender': 'MALE', 'id': 5804, 'is_completed': True, 'name': 'Krisna', 'photo_url': '', 'profile_completions': {'section_validations': [], 'validation_message': 'Data anda belum lengkap', 'validation_percentage': 88}}, 'client_attendance': '', 'description': 'test', 'id': 18569, 'is_client_rated': False, 'last_call': '2024-02-26T03:39:48.000000Z', 'lawyer': {'affidavit_date': '2015-12-31T17:00:00.000000Z', 'agency_city': {'id': 3171, 'name': 'JAKARTA SELATAN'}, 'agency_name': 'Perqara', 'agency_province': {'id': 31, 'name': 'DKI JAKARTA'}, 'avg_rating': 5, 'avg_ratings': '5.0', 'city': {'id': 3171, 'name': 'JAKARTA SELATAN'}, 'description': 'Akun ini bertujuan khusus untuk Survei Klien yang memiliki kendala Teknis pada Platform Perqara. Di luar dari Responden maka akan Kami tolak.', 'educations': [{'degree': 'S1', 'institution_id': 189, 'institution_name': 'Duta Bangsa University'}], 'gender': 'MALE', 'id': 324, 'is_busy': True, 'is_online': True, 'is_probono': True, 'name': 'Perqara Testing', 'photo_url': 'http

In [12]:
# Assuming df is your DataFrame and 'json_column' is the column containing JSON data
normalized_df = pd.json_normalize(df['message'].apply(ast.literal_eval), max_level=5)

In [13]:
normalized_df

,event,data.booking.booking_date,data.booking.booking_time,data.booking.bookingable_id,data.booking.bookingable_type,data.booking.consultation_id,data.booking.created_at,data.booking.duration,data.booking.id,data.booking.parent_id,...,data.room_expired_at,data.room_key,data.skill.id,data.skill.name,data.status,data.stop_time,data.summary_sent_at,data.total_price,data.waiting_expired_at,data.current_time
0,LAWYER:NEW_CONSULTATION-TRYING-0,2024-02-26,03:39:00,13671,AvailabilityInstant,18569,2024-02-26T03:39:39.000000Z,30,18411,0,...,,eyJpdiI6IlVRS2J0eUlFMUl3alpHTWZrUzJwQ3c9PSIsInZhbHVlIjoiS3duMXRud0FWQjhsdEF3STA5RTloWTM3RGZyTHIxb2I2dlo1NUU5RW00bm5ldUNJYVVURkloaUI0cFJKcmkvRjNuVTJmMEFINFk3elE1eVBYWWJEVmVWdy9BdWQyYkhPS2RydURueDJaZlVDL0VqR2RQQnd1L3plNTV0bWJEcmYxOFgybytRaWEvQkNLUzVFeDRpcHhRPT0iLCJtYWMiOiJmYTMxYzhmN2JjYzcyYjE5YzRkNzE4YWZmNzAzMDkwNmMzYmIxMGExZmE3MmY1NmNkOGNiYTZjMmQwYzAyNjcwIiwidGFnIjoiIn0=,2,Pidana,WAITING_FOR_APPROVAL,,,0,2024-02-26T03:42:45.000000Z,NaN
1,LAWYER:JOIN_ROOM-TRYING-0,2024-02-26,03:39:00,13671,AvailabilityInstant,18569,2024-02-26T03:39:39.000000Z,30,18411,0,...,2024-02-26T04:09:48.000000Z,eyJpdiI6IlVRS2J0eUlFMUl3alpHTWZrUzJwQ3c9PSIsInZhbHVlIjoiS3duMXRud0FWQjhsdEF3STA5RTloWTM3RGZyTHIxb2I2dlo1NUU5RW00bm5ldUNJYVVURkloaUI0cFJKcmkvRjNuVTJmMEFINFk3elE1eVBYWWJEVmVWdy9BdWQyYkhPS2RydURueDJaZlVDL0VqR2RQQnd1L3plNTV0bWJEcmYxOFgybytRaWEvQkNLUzVFeDRpcHhRPT0iLCJtYWMiOiJmYTMxYzhmN2JjYzcyYjE5YzRkNzE4YWZmNzAzMDkwNmMzYmIxMGExZmE3MmY1NmNkOGNiYTZjMmQwYzAyNjcwIiwidGFnIjoiIn0=,2,Pidana,ON_PROCESS,,,0,2024-02-26T03:42:48.000000Z,NaN
2,CLIENT:JOIN_ROOM-TRYING-0,2024-02-26,03:39:00,13671,AvailabilityInstant,18569,2024-02-26T03:39:39.000000Z,30,18411,0,...,2024-02-26T04:09:48.000000Z,eyJpdiI6IlVRS2J0eUlFMUl3alpHTWZrUzJwQ3c9PSIsInZhbHVlIjoiS3duMXRud0FWQjhsdEF3STA5RTloWTM3RGZyTHIxb2I2dlo1NUU5RW00bm5ldUNJYVVURkloaUI0cFJKcmkvRjNuVTJmMEFINFk3elE1eVBYWWJEVmVWdy9BdWQyYkhPS2RydURueDJaZlVDL0VqR2RQQnd1L3plNTV0bWJEcmYxOFgybytRaWEvQkNLUzVFeDRpcHhRPT0iLCJtYWMiOiJmYTMxYzhmN2JjYzcyYjE5YzRkNzE4YWZmNzAzMDkwNmMzYmIxMGExZmE3MmY1NmNkOGNiYTZjMmQwYzAyNjcwIiwidGFnIjoiIn0=,2,Pidana,ON_PROCESS,,,0,2024-02-26T03:42:48.000000Z,NaN
3,LAWYER:JOIN_ROOM-TRYING-1,2024-02-26,03:39:00,13671,AvailabilityInstant,18569,2024-02-26T03:39:39.000000Z,30,18411,0,...,2024-02-26T04:09:48.000000Z,eyJpdiI6IlVRS2J0eUlFMUl3alpHTWZrUzJwQ3c9PSIsInZhbHVlIjoiS3duMXRud0FWQjhsdEF3STA5RTloWTM3RGZyTHIxb2I2dlo1NUU5RW00bm5ldUNJYVVURkloaUI0cFJKcmkvRjNuVTJmMEFINFk3elE1eVBYWWJEVmVWdy9BdWQyYkhPS2RydURueDJaZlVDL0VqR2RQQnd1L3plNTV0bWJEcmYxOFgybytRaWEvQkNLUzVFeDRpcHhRPT0iLCJtYWMiOiJmYTMxYzhmN2JjYzcyYjE5YzRkNzE4YWZmNzAzMDkwNmMzYmIxMGExZmE3MmY1NmNkOGNiYTZjMmQwYzAyNjcwIiwidGFnIjoiIn0=,2,Pidana,ON_PROCESS,,,0,2024-02-26T03:42:48.000000Z,NaN
4,LAWYER:JOIN_ROOM-TRYING-2,2024-02-26,03:39:00,13671,AvailabilityInstant,18569,2024-02-26T03:39:39.000000Z,30,18411,0,...,2024-02-26T04:09:48.000000Z,eyJpdiI6IlVRS2J0eUlFMUl3alpHTWZrUzJwQ3c9PSIsInZhbHVlIjoiS3duMXRud0FWQjhsdEF3STA5RTloWTM3RGZyTHIxb2I2dlo1NUU5RW00bm5ldUNJYVVURkloaUI0cFJKcmkvRjNuVTJmMEFINFk3elE1eVBYWWJEVmVWdy9BdWQyYkhPS2RydURueDJaZlVDL0VqR2RQQnd1L3plNTV0bWJEcmYxOFgybytRaWEvQkNLUzVFeDRpcHhRPT0iLCJtYWMiOiJmYTMxYzhmN2JjYzcyYjE5YzRkNzE4YWZmNzAzMDkwNmMzYmIxMGExZmE3MmY1NmNkOGNiYTZjMmQwYzAyNjcwIiwidGFnIjoiIn0=,2,Pidana,ON_PROCESS,,,0,2024-02-26T03:42:48.000000Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,LAWYER:JOIN_ROOM-TRYING-3,2024-02-26,16:28:00,13684,AvailabilityInstant,18739,2024-02-26T16:28:54.000000Z,30,18578,0,...,2024-02-26T16:59:36.000000Z,eyJpdiI6IjR4QmxLR21jYUIvRXd2eWI5dkllZHc9PSIsInZhbHVlIjoiY1lGck9BWktnengvSEFKcmNOM3ZEcWExVjA2dk53ZEVqNnE0cnl4aTJMN1VYODZBajM2dCs0WXNFM1VBckNqcUgrSlNMdi93RUI5aldOR01ncWM2SGdIN25WU285RzlLWWVxcWVLejlKYTYrT3h1SkhQMkFUakN2MUpjYm5sUUU2MnhTOHcvYXk2WGlBZ2UxaXZQRjJRPT0iLCJtYWMiOiIyOGZmZTk4MGUyNGQ5ZWY1YjNkYzY4N2I3NmFlZDVmYWU1ZjBjY2ZjNzM2MmNkMTVmZjRhMTBhMjY5ZGZiYjAzIiwidGFnIjoiIn0=,2,Pidana,ON_PROCESS,,,0,2024-02-26T16:32:36.000000Z,NaN
865,CLIENT:LAWYER_REJECT_CONSULTA

In [ ]:
normalized_df.columns

Index(['event', 'data.booking.booking_date', 'data.booking.booking_time',
       'data.booking.bookingable_id', 'data.booking.bookingable_type',
       'data.booking.consultation_id', 'data.booking.created_at',
       'data.booking.duration', 'data.booking.id', 'data.booking.parent_id',
       'data.booking.status', 'data.booking.updated_at',
       'data.client.address.address', 'data.client.address.city_id',
       'data.client.address.city_name', 'data.client.address.id',
       'data.client.address.postal_code_id', 'data.client.address.status',
       'data.client.birth_date', 'data.client.gender', 'data.client.id',
       'data.client.is_completed', 'data.client.name', 'data.client.photo_url',
       'data.client.profile_completions.section_validations',
       'data.client.profile_completions.validation_message',
       'data.client.profile_completions.validation_percentage',
       'data.client_attendance', 'data.description', 'data.id',
       'data.is_client_rated', 'data.last

In [ ]:
df['event_detail'] = normalized_df['event'].to_numpy()

In [ ]:
df

,event,level,message,timestamp,event_detail
28,nan,info,"{'data': {'booking': {'booking_date': '2024-02-17', 'booking_time': '17:03:00', 'bookingable_id': 13372, 'bookingable_type': 'AvailabilityInstant', 'consultation_id': 17575, 'created_at': '2024-02-17T17:03:47.000000Z', 'duration': 30, 'id': 17434, 'parent_id': 0, 'status': 0, 'updated_at': '2024-02-17T17:03:47.000000Z'}, 'client': {'address': {'address': '', 'city_id': 3604, 'city_name': 'SERANG', 'id': 7996, 'postal_code_id': 0, 'status': 1}, 'birth_date': '2002-05-04', 'gender': 'MALE', 'id': 15051, 'is_completed': True, 'name': 'Sanam', 'photo_url': '', 'profile_completions': {'section_validations': [], 'validation_message': 'Data anda belum lengkap', 'validation_percentage': 75}}, 'client_attendance': '', 'description': 'saya ingin membuat sertifikat tanah milik alm. Orang tua saya tapi tanah itu sebelumnya berstatus akte hibah. Bagaimana yah caranya?', 'id': 17575, 'is_client_rated': False, 'last_call': '2024-02-17T17:03:48.000000Z', 'lawyer': {'affidavit_date': '2020-08-30T17:00:00.000000Z', 'agency_city': {'id': 3201, 'name': 'BOGOR'}, 'agency_name': 'PBH Peradi', 'agency_province': {'id': 32, 'name': 'JAWA BARAT'}, 'avg_rating': 5, 'avg_ratings': '5.0', 'city': {'id': 3201, 'name': 'BOGOR'}, 'description': ""Genuari Waruwu, S.H. merupakan Advokat dari PERADI dan juga menjabat sebagai Pengurus Bidang Kerjasama dan Hubungan Antar Lembaga Dewan Pimpinan Cabang atau DPC PERADI CIBINONG, Bogor - Jawa Barat, sebagai Ketua Koordinator Bidang Kampanye dan Kemitraan pada Pusat Bantuan Hukum atau PBH PERADI CIBINONG, Bogor - Jawa Barat, serta sebagai Kepala Divisi Inovatif dan Kreatif pada Young Lawyer's Committtee atau YLC DPC PERADI CIBINONG, Bogor - Jawa Barat."", 'educations': [{'degree': 'S1', 'institution_id': 255, 'institution_name': 'Universitas Wiraswasta Indonesia'}], 'gender': 'MALE', 'id': 294, 'is_busy': False, 'is_online': True, 'is_probono': True, 'name': 'Genuari Waruwu, S.H.', 'photo_url': 'https://storage.googleapis.com/perqara-public/lawyer/photo_1703131378', 'price': 0, 'skill_ids': [2, 3, 4, 5, 6], 'skills': [{'id': 2, 'name': 'Pidana'}, {'id': 3, 'name': 'Perdata'}, {'id': 4, 'name': 'Ketenagakerjaan'}, {'id': 5, 'name': 'Perkawinan & Perceraian'}, {'id': 6, 'name': 'Pertanahan'}], 'slug': 'Genuari Waruwu, S.H.', 'total_consultations': 0, 'year_exp': 3}, 'lawyer_approved_at': '', 'lawyer_attendance': '', 'room_expired_at': '', 'room_key': 'eyJpdiI6ImpzTEpOcWJPbTRNWERBcUhyK1dRbWc9PSIsInZhbHVlIjoiejIzVCs0QmViVDVucTJGY0ttRTJVZGY4RTVIOGNCSTlxTzRNRkpkZVJVOEowRzZNWXB3b2FjNExxK3ZlTkVFR1RKeG1hMmNYZG1YTEJqYUpxd3hwM3ZVdWVPOVdOTUJaaGZLZ2VQUmp5KzVUTGx0NkdUOGY4RERNY2FPb3FscXpXNUkzdUJ5QnlFaWx6R0dWaXpFK2lnPT0iLCJtYWMiOiIyMDBhNjUzZThiMTlhMGFmZGYwZGQ1NzgwNmFhNTRiNDBmNjM1NmVkYWYzNjgwZWIyYzBhNmM2MzlkNjczYzU1IiwidGFnIjoiIn0=', 'skill': {'id': 6, 'name': 'Pertanahan'}, 'status': 'WAITING_FOR_APPROVAL', 'stop_time': '', 'summary': {}, 'summary_sent_at': '', 'total_price': 0, 'waiting_expired_at': '2024-02-17T17:06:48.000000Z'}, 'event': 'LAWYER:NEW_CONSULTATION-TRYING-0'}",2024-02-18 00:03:49,LAWYER:NEW_CONSULTATION-TRYING-0
36,nan,info,"{'data': {'booking': {'booking_date': '2024-02-17', 'booking_time': '17:03:00', 'bookingable_id': 13372, 'bookingable_type': 'AvailabilityInstant', 'consultation_id': 17575, 'created_at': '2024-02-17T17:03:47.000000Z', 'duration': 30, 'id': 17434, 'parent_id': 0, 'status': 2, 'updated_at': '2024-02-17T17:03:59.000000Z'}, 'client': {'address': {'address': '', 'city_id': 3604, 'city_name': 'SERANG', 'id': 7996, 'postal_code_id': 0, 'status': 1}, 'birth_date': '2002-05-04', 'gender': 'MALE', 'id': 15051, 'is_completed': True, 'name': 'Sanam', 'photo_url': '', 'profile_completions': {'section_validations': [], 'validation_message': 'Data anda belum lengkap', 'validation_percentage': 75}}, 'client_attendance': '', 'description': 'saya ingin membuat sertifikat tanah milik alm. Orang tua saya tapi tanah itu sebelumnya berstatus akte hibah. Bagaim